`NstepRNN`系はcudnnを使っているため中断すると確実にカーネルが落ちる

In [1]:
%matplotlib inline
import sys
sys.path.append("../")
import pickle
import numpy as np
from tqdm import tqdm_notebook
from prefetch_generator import BackgroundGenerator
from matplotlib import pylab as plt
from IPython.display import clear_output
import os
from joblib import Parallel, delayed
from tqdm import tqdm
import nltk
from glob import glob
from joblib import Parallel, delayed
from collections import Counter
from layers import *
from utils import *
from models import *
import json
import tensorflow as tf
tfe = tf.contrib.eager 
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1"))
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
tf.enable_eager_execution(config=config)

In [2]:
trn_data = read_pickle("../dataset/nl/train.pkl")
vld_data = read_pickle("../dataset/nl/valid.pkl")
tst_data = read_pickle("../dataset/nl/test.pkl")
code_i2w = read_pickle("../dataset/code_i2w.pkl")
code_w2i = read_pickle("../dataset/code_w2i.pkl")
nl_i2w = read_pickle("../dataset/nl_i2w.pkl")
nl_w2i = read_pickle("../dataset/nl_w2i.pkl")

In [3]:
trn_x, trn_y_raw = zip(*trn_data.items())
vld_x, vld_y_raw = zip(*vld_data.items())
tst_x, tst_y_raw = zip(*tst_data.items())

In [4]:
trn_y = [[nl_w2i[t] if t in nl_w2i.keys() else nl_w2i["<UNK>"] for t in l] for l in trn_y_raw]
vld_y = [[nl_w2i[t] if t in nl_w2i.keys() else nl_w2i["<UNK>"] for t in l] for l in vld_y_raw]
tst_y = [[nl_w2i[t] if t in nl_w2i.keys() else nl_w2i["<UNK>"] for t in l] for l in tst_y_raw]

In [6]:
# setting model
# model = Model(512, 512, 512, len(code_w2i), len(nl_w2i))
epochs = 20
batch_size = 64
# checkpoint_dir = "/mnt/hdd/models/summarize/childsum_alldata_1layer"
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# root = tfe.Checkpoint(model=model)
history = {"loss":[], "loss_val":[], "bleu": []}

In [7]:
# Setting Data Generator

trn_gen = Datagen_tree(trn_x, trn_y, batch_size, code_w2i, nl_i2w, train=True)
vld_gen = Datagen_tree(vld_x, vld_y, batch_size, code_w2i, nl_i2w, train=False)
tst_gen = Datagen_tree(tst_x, tst_y, batch_size, code_w2i, nl_i2w, train=False)

In [13]:
gen = trn_gen(0)
for x, y, _, _ in gen:
    x, y, _, _ = x, y, _, _
    break

In [17]:
class BiLSTM(tf.keras.Model):
    def __init__(self, dim):
        super(BiLSTM, self).__init__()
        self.dim = dim
        self.LSTM_f = tf.keras.layers.CuDNNLSTM()

class ShidoTreeLSTM(tf.keras.Model):
    def __init__(self, dim_in, dim_out):
        super(ShidoTreeLSTM, self).__init__()
        self.dim_in = dim_in
        self.dim_out = dim_out
        self.U_f = tf.keras.layers.Dense(dim_out, use_bias=False)
        self.U_iuo = tf.keras.layers.Dense(dim_out * 3, use_bias=False)
        self.W = tf.keras.layers.Dense(dim_out * 4, use_bias=False)
        self.h_init = tfe.Variable(
            tf.get_variable("h_init", [1, dim_out], tf.float32, initializer=he_normal()))
        self.c_init = tf.constant(0., tf.float32, [1, self.dim_out])

    @staticmethod
    def get_nums(roots):
        res = [[x.num for x in n.children] if n.children != [] else [0] for n in roots]
        max_len = max([len(x) for x in res])
        res = tf.keras.preprocessing.sequence.pad_sequences(
            res, max_len, padding="post", value=-1.)
        return tf.constant(res, tf.int32)

    def call(self, roots):
        depthes = [x[1] for x in sorted(depth_split_batch2(
            roots).items(), key=lambda x:-x[0])]  # list of list of Nodes
        indices = [self.get_nums(nodes) for nodes in depthes]

        h_tensor = self.h_init
        c_tensor = self.c_init
        for indice, nodes in zip(indices, depthes):
            x = tf.stack([node.h for node in nodes])  # [nodes, dim_in]
            h_tensor, c_tensor = self.apply(x, h_tensor, c_tensor, indice, nodes)
            h_tensor = tf.concat([self.h_init, h_tensor], 0)
            c_tensor = tf.concat([self.c_init, c_tensor], 0)
        return depthes[-1]

    def apply(self, x, h_tensor, c_tensor, indice, nodes):

        mask_bool = tf.not_equal(indice, -1.)
        mask = tf.cast(mask_bool, tf.float32)  # [batch, child]

        h = tf.gather(h_tensor, tf.where(mask_bool,
                                         indice, tf.zeros_like(indice)))  # [nodes, child, dim]
        c = tf.gather(c_tensor, tf.where(mask_bool,
                                         indice, tf.zeros_like(indice)))
        h_sum = tf.reduce_sum(h * tf.expand_dims(mask, -1), 1)  # [nodes, dim_out]

        W_x = self.W(x)  # [nodes, dim_out * 4]
        W_f_x = W_x[:, :self.dim_out * 1]  # [nodes, dim_out]
        W_i_x = W_x[:, self.dim_out * 1:self.dim_out * 2]
        W_u_x = W_x[:, self.dim_out * 2:self.dim_out * 3]
        W_o_x = W_x[:, self.dim_out * 3:]

        branch_f_k = tf.reshape(self.U_f(tf.reshape(h, [-1, h.shape[-1]])), h.shape)
        branch_f_k = tf.sigmoid(tf.expand_dims(W_f_x, 1) + branch_f_k)
        branch_f = tf.reduce_sum(branch_f_k * c * tf.expand_dims(mask, -1), 1)  # [node, dim_out]

        branch_iuo = self.U_iuo(h_sum)  # [nodes, dim_out * 3]
        branch_i = tf.sigmoid(branch_iuo[:, :self.dim_out * 1] + W_i_x)   # [nodes, dim_out]
        branch_u = tf.tanh(branch_iuo[:, self.dim_out * 1:self.dim_out * 2] + W_u_x)
        branch_o = tf.sigmoid(branch_iuo[:, self.dim_out * 2:] + W_o_x)

        new_c = branch_i * branch_u + branch_f  # [node, dim_out]
        new_h = branch_o * tf.tanh(new_c)  # [node, dim_out]

        for n, c, h in zip(nodes, new_c, new_h):
            n.c = c
            n.h = h

        return new_h, new_c


<tf.Tensor: id=705, shape=(64, 31), dtype=int32, numpy=
array([[ 1377,  6563, 24460, ...,    -1,    -1,    -1],
       [ 1377,  4642, 21914, ...,    -1,    -1,    -1],
       [ 1377,  9943, 13533, ...,    -1,    -1,    -1],
       ...,
       [ 1377, 11351, 20333, ...,    -1,    -1,    -1],
       [ 1377,  6563, 28079, ...,    -1,    -1,    -1],
       [ 1377,  5355, 28045, ...,    -1,    -1,    -1]], dtype=int32)>

In [9]:
# training
for epoch in range(epochs):
    
    # train
    loss_tmp = []
    t = tqdm(trn_gen(epoch))
    for x, y, _, _ in t:
        loss_tmp.append(model.train_on_batch(x, y))
        t.set_description("epoch:{:03d}, loss = {}".format(epoch + 1, np.mean(loss_tmp)))
    history["loss"].append(np.sum(loss_tmp) / len(t))
    
    loss_tmp = []
    t = tqdm(vld_gen(epoch))
    for x, y, _, _ in t:
        loss_tmp.append(model.evaluate_on_batch(x, y))
        t.set_description("epoch:{:03d}, loss_val = {}".format(epoch + 1, np.mean(loss_tmp)))
    history["loss_val"].append(np.sum(loss_tmp) / len(t))
    
    # checkpoint
    if history["loss_val"][-1] == min(history["loss_val"]):
        checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
        root.save(file_prefix=checkpoint_prefix)
    
    # print
    clear_output()
    for key, val in history.items():
        if "loss" in key:
            plt.plot(val, label=key)
    plt.legend()
    plt.show()

with open(os.path.join(checkpoint_dir, "history.json"), "w") as f:
    json.dump(history, f)

epoch:001, loss = 6.775298118591309:   1%|          | 28/2382 [00:42<1:01:59,  1.58s/it]

KeyboardInterrupt: 

In [ ]:
root.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
preds = []
trues = []
for x, y, _, y_raw in tqdm(tst_gen(0)):
    res = model.translate(x, nl_i2w, nl_w2i)
    preds += res
    trues += [s[1:-1] for s in y_raw]

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [ ]:
import math
def ngram(words, n):
    return list(zip(*(words[i:] for i in range(n))))

def bleu4(true, pred):
    c = len(pred)
    r = len(true)
    bp = 1. if c > r else np.exp(1 - r / c)
    score = 0
    for i in range(1, 5):
        true_ngram = set(ngram(true, i))
        pred_ngram = ngram(pred, i)
        length = float(len(pred_ngram)) + 1e-10
        count = sum([1. if t in true_ngram else 0. for t in pred_ngram])
        score += math.log(1e-10 + (count / length))
    score = math.exp(score * .25)
    bleu = bp * score
    return bleu

In [ ]:
bleus = Parallel(n_jobs=-1)(delayed(bleu4)(t, p) for t, p in tqdm(list(zip(trues, preds))))

In [ ]:
np.mean(bleus)